In [7]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
#Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '94b2912e-ed50-4320-83cf-e49bfb331682',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)


df = pd.json_normalize(data['data'])

df["Day"] = pd.to_datetime(df["last_updated"]).dt.date

df["Coin"] = df["symbol"]

df["Price"] = round(df["quote.USD.price"],2)

df["Price_change_24h"] = round(df["quote.USD.percent_change_24h"],2)

df["Price_change_90d"] = round(df["quote.USD.percent_change_90d"],2)

df["Price_1h"] = abs(round(-((df["quote.USD.price"]*df["quote.USD.percent_change_1h"]+100)/100)+df["quote.USD.price"],2))

df["Price_1d"] = abs(round(-((df["quote.USD.price"]*df["quote.USD.percent_change_1h"]+100)/100)+df["quote.USD.price"],2))

df["Price_1w"] = abs(round(-((df["quote.USD.price"]*df["quote.USD.percent_change_7d"]+100)/100)+df["quote.USD.price"],2))

df["Price_30day"] = abs(round(-((df["quote.USD.price"]*df["quote.USD.percent_change_30d"]+100)/100)+df["quote.USD.price"],2))

df["Price_90d"] = abs(round(-((df["quote.USD.price"]*df["quote.USD.percent_change_90d"]+100)/100)+df["quote.USD.price"],2))

df['Market_cap'] = round(df["quote.USD.fully_diluted_market_cap"],0)


df2 = df.iloc[:,36:].head(200)

df2.to_csv(r'C:\Users\Admin\Documents\crytoApp\edit.csv',index=False)
